In [22]:
import pandas as pd
from datetime import datetime

data = pd.read_csv('./transactionsTest2.csv').to_dict(orient='records') # Converts csv to pandas dataframe and then to list of dictionaries
print(data)

cardData = pd.read_csv('./creditCardList.csv').to_dict(orient='records') 
print(cardData)

[{'Date': '2023-01-01', 'Merchant': 'Aldi', 'Category': 'grocery', 'Amount': 97.2}, {'Date': '2023-01-01', 'Merchant': 'Lyft', 'Category': 'transportation', 'Amount': 36.39}, {'Date': '2023-01-01', 'Merchant': 'Radisson', 'Category': 'hotel', 'Amount': 348.74}, {'Date': '2023-01-01', 'Merchant': 'Chipotle', 'Category': 'restaurant', 'Amount': 53.39}, {'Date': '2023-01-03', 'Merchant': 'The Cheesecake Factory', 'Category': 'restaurant', 'Amount': 61.36}, {'Date': '2023-01-05', 'Merchant': 'Subway', 'Category': 'restaurant', 'Amount': 101.11}, {'Date': '2023-01-08', 'Merchant': 'Costco', 'Category': 'grocery', 'Amount': 222.14}, {'Date': '2023-01-08', 'Merchant': "McDonald's", 'Category': 'restaurant', 'Amount': 147.29}, {'Date': '2023-01-10', 'Merchant': 'Starbucks', 'Category': 'restaurant', 'Amount': 27.75}, {'Date': '2023-01-11', 'Merchant': 'Olive Garden', 'Category': 'restaurant', 'Amount': 118.93}, {'Date': '2023-01-13', 'Merchant': 'Taco Bell', 'Category': 'restaurant', 'Amount':

In [23]:
totals = {
    'grocery': {'amount': 0, 'transactions': 0, 'days_difference': 0, 'last_date': None},
    'hotel': {'amount': 0, 'transactions': 0, 'days_difference': 0, 'last_date': None},
    'transportation': {'amount': 0, 'transactions': 0, 'days_difference': 0, 'last_date': None},
    'restaurant': {'amount': 0, 'transactions': 0, 'days_difference': 0, 'last_date': None}
}

for transaction in data:
    category = transaction['Category']
    if category in totals:
        totals[category]['amount'] += transaction['Amount']
        totals[category]['transactions'] += 1

        current_date = datetime.strptime(transaction['Date'], '%Y-%m-%d')

        if totals[category]['last_date'] is not None:
            days_diff = (current_date - totals[category]['last_date']).days
            totals[category]['days_difference'] += days_diff
        
        totals[category]['last_date'] = current_date

for category, stats in totals.items():
    print(f"\n{category}:")
    print(f"  Total Amount: ${stats['amount']:,.2f}")
    print(f"  Total Transactions: {stats['transactions']}")
    
    if stats['transactions'] > 1:
        avgDays = stats['days_difference'] / (stats['transactions'] - 1)
        print(f"  Average Days Between Transactions: {avgDays:.2f}")
    else:
        print(f"  Average Days Between Transactions: N/A (only 1 transaction)")


grocery:
  Total Amount: $17,579.14
  Total Transactions: 105
  Average Days Between Transactions: 7.00

hotel:
  Total Amount: $5,584.12
  Total Transactions: 17
  Average Days Between Transactions: 43.81

transportation:
  Total Amount: $5,612.56
  Total Transactions: 59
  Average Days Between Transactions: 12.55

restaurant:
  Total Amount: $29,541.94
  Total Transactions: 366
  Average Days Between Transactions: 1.99


In [24]:
import math

total_amount_all = sum(v['amount'] for v in totals.values())
max_transactions = max(v['transactions'] for v in totals.values())

normalized = {}

for category, stats in totals.items():
    amount = stats['amount']
    transactions = stats['transactions']

    if transactions > 1:
        avg_days = stats['days_difference'] / (transactions - 1)
    else:
        avg_days = float('inf')

    # Spending Score (Percentage of total spending)
    # Values closer to 1 mean this category dominates overall spending
    S = amount / total_amount_all if total_amount_all > 0 else 0 

    # Frequency (Helps rule out micro-transactions/outliers)
    # Values closer to 1 mean the user spends in this category frequently relative to other categories
    F = math.log(1 + transactions) / math.log(1 + max_transactions) if max_transactions > 0 else 0 
    
    # T = Timing Consistency Score
    # Timing (Values closer to 1 means this transaction is consistent and scheduled)
    T = 1 / (1 + avg_days) if avg_days != float('inf') else 0 

    engagementScore = (
        0.55 * S +
        0.25 * F +
        0.20 * T
    )

    normalized[category] = {
        'S': S,
        'F': F,
        'T': T,
        'score': engagementScore,
        'amount': amount
    }

normalized

{'grocery': {'S': 0.3014371608237354,
  'F': 0.7896957399229252,
  'T': 0.125,
  'score': 0.3882143734337858,
  'amount': 17579.140000000007},
 'hotel': {'S': 0.09575333483316231,
  'F': 0.48944871326527645,
  'T': 0.022315202231520222,
  'score': 0.17948955292086244,
  'amount': 5584.120000000001},
 'transportation': {'S': 0.09624100788507649,
  'F': 0.6933266186848345,
  'T': 0.07379134860050891,
  'score': 0.24102247872810248,
  'amount': 5612.5599999999995},
 'restaurant': {'S': 0.5065684964580258,
  'F': 1.0,
  'T': 0.3348623853211009,
  'score': 0.5955851501161344,
  'amount': 29541.940000000002}}

In [ ]:
def recommendCard(cardData, normalized): # Returns top 3 cards
    recommendations = []

    categoryToCardRate = {
        'grocery': 'grocery_rate',
        'restaurant': 'dining_rate',
        'hotel': 'travel_rate',
        'transportation': 'base_rate'
    }

    for card in cardData:
        score = 0

        # Score is calculated as the sum across all categories of (engagement * amountSpent * cardReward) with rewards capped applied to each category if applicable minus any annual fees
        for category, data in normalized.items():
            rewardRate = card[categoryToCardRate[category]]
            # Amount * Card rewards * Engagement
            score += data['amount'] * rewardRate * data['score']

            # Spending cap if applicable
            if card['spend_cap'] > 0:
                score = min(score, card['spend_cap'])

        # Subtract annual fee
        score -= card['annual_fee']

        recommendations.append({'card_name': card['name'], 'score': score})

    # Sort descending by score
    recommendations.sort(key=lambda x: x['score'], reverse=True)
    
    return recommendations


In [26]:
topCards = recommendCard(cardData, normalized)

# Show top 3 recommendations
for i, card in enumerate(topCards[:3], 1):
    print(f"{i}. {card['card_name']}: Score = {card['score']:.2f}")

1. Chase_Freedom_Flex: Score = 932.71
2. CapitalOne_SavorOne: Score = 796.22
3. Amex_Gold: Score = 770.36


In [ ]:
# Rotating Categories

In [ ]:
# Using Special Portals for More Rewards